# Enzyme Commission Class with Ligands


`````{admonition} Overview
:class: overview

Questions

* How are enzymes classified?

* How can I search the PDB for ligands that bind to a specific enzyme class?


Objectives

* Understand the hierarchical classification of enzymes using the Enzyme Commission (EC) system.

* Use the RCSB Search API to find ligands that bind to a specific enzyme class.

`````

Enzymes are biological catalysts and most enzymes are proteins (at least that's our current thinking). 
To systematize the study of enzymes, IUPAC (the International Union of Pure and Applied Chemistry) has organized enzymes in a hierarchical class structure, with 7 top level classes and a total of 4 levels in the hierarchy. 

These classes are called the Enzyme Commission (EC) classes, and the hierarchy is called the EC system.
Each enzyme is assigned an EC number based on the type of reaction it catalyzes.
In this notebook, we will use the RCSB Search API to find ligands that bind to a specific enzyme class.

## The 7 Enzyme Classes

The [BRENDA Enzyme Database](https://www.brenda-enzymes.org/) contains detailed information about enzymes and includes a [browser tree for enzyme classes](https://www.brenda-enzymes.org/ecexplorer.php?browser=1). The 7 major classes of enzymes on the top level of the tree are
1. Oxidoreductases
1. Transferases
1. Hydrolases
1. Lyases
1. Isomerases
1. Ligases
1. Translocases

On the [BRENDA Home Page](https://www.brenda-enzymes.org/index.php), you can search for an enzyme by name and find its Enzyme Commision (EC) number, and a lot more detail as well. 
The first image below is the search for the enzyme HMG CoA Reductase, the control point for cholesterol synthesis. 
The second image shows the initial results page, which includes the EC number. 

![The BRENDA Home Page](images/BRENDA_Home_Page_search_for_HMG-CoA_Reductase.png)

![BRENDA Search Results](images/BRENDA_HMG-CoA_Reductase_search.png)

In this notebook, we will focus on trypsin, an enzyme produced by your pancreas that helps to digest proteins in your small intestine. It is a member of family called serine hydrolases. Members of this family use water to break bonds in proteins, lipids and carbohydrates. They are involved in digestion, cell signalling and blood clotting. The EC# for trypsin is 3.4.21.4. Here is the meaning of each level of the hierarchy:

* 3 - Hydrolase
* 3.4 - acting on peptide bonds
* 3.4.21 - serine endopeptidase
* 3.4.21.4 - trypsin

This notebook is intended to help users find ligands for use with docking studies. 
We are looking for ligands that will bind to trypsin, with the intention of seeing if they will bind to other serine hydrolases that might be interesting. Here are the steps we will follow in the process, all of which will employ the `rcsbsearchapi` package. 

1. Find PDB structures of a given Enzyme Commission class.
2. Select those structures that contain bound small molecules with molecular weights between 300 and 600.
3. Output a list of those ligands
4. Save the ligand structures to the "ligands_for_EC_class_#" folder.

Text (actually markdown) cells will be inserted to explain each step.

### Libraries for the IQB workshop

| Library         | abbreviation | Purpose |
|:-------------|:---------:|:------------|
| rcsbsearchapi | N/A      | functions for searching the Protein Data Bank based on the mmCIF dictionary |
| os           | N/A      | operating system functions - handling file paths and directories. |
| requests     | N/A  | access APIs for databases |
| rdkit | rdkit | an open source github repository of cheminformatics software|
| rdkit.Chem | Chem | a subset of rdkit that supports file string to structure conversions |
| rdkit.Chem.AllChem | AllChem | a subset of rdkit.Chem that supports energy optimization |
| rdkit.Chem.Draw | Draw | a subset of rdkit that supports chemical drawing in Python |
| vina | vina | AutoDock Vina software for Python and Jupyter notebooks |



In [1]:
# Import the components of rcsbsearchapi needed for this search
from rcsbsearchapi import rcsb_attributes as attrs

## Making queries
To make a query with `rcsbsearchapi` you first must know what you are looking for. I find it helpful to actually write this out by hand sometimes. Here are the characteristics I am looking for in ligands that bind to a specific Enzyme Commission Class of a protein.
- EC Class. I will focus on the EC class for trypsin, 3.4.21.4, but any class should work.
- Ligands. I am looking for ligands that are larger than a single atom (e.g., potassium ion) or a buffer molecule (phosphate), but of a size that consists of 10-30 heavy atoms, so I will aim for a molecular weight between 300 and 800.

Please note that you can use this interface to search for dozens of attributes associated with a PDB entry. The attribute that we will use to look for proteins that have the EC# = 3.4.21.4 is `rcsb_polymer_entity.rcsb_ec_lineage.id`. Other searchable attributes include the abbreviated journal title for the primary citation, `rcsb_primary_citation.rcsb_journal_abbrev`, the method used to determine the structure `exptl.method`, or specific molecules that are part of PDB entries `pdbx_reference_molecule.class'.  

In [2]:
# There will be three components to the query, which will be labeled q1, q2 and q3.

ECnumber = "3.4.21.4"     # We will use this variable again later

q1 = attrs.rcsb_polymer_entity.rcsb_ec_lineage.id == ECnumber  # looking for trypsins
q2 = attrs.chem_comp.formula_weight >= 300                       # setting the lower limit for molecular weight
q3 = attrs.chem_comp.formula_weight <= 800                       # setting the upper limit for molecular weight

query = q1 & q2 & q3              # combining the three queries into one

resultL = list(query())           # assign the results of the query to a list variable

print(resultL[0:10])              # list the first 10 results

len(resultL)

['1AQ7', '1AUJ', '1AZ8', '1BJV', '1BTW', '1BTX', '1BTZ', '1C1S', '1C1T', '1C2D']


180

### Finding the ligands

This query provided the list of the PDB entries for trypsins (EC # 3.4.21.4) that contain ligands between 300 and 800 molecular weight.
We printed the first 10 of these results using `print(resultL[0:10])`. 

Why would we choose to have ligands with molecular weights between 300 and 800? We are interested in molecules that are large enough to bind to and fill up the active site of trypsin. Small molecules with molecular weights less than 300 are likely to be individual ions (K<sup>+</sup> or Na<sup>+</sup>). Molecules with molecular weights greater than 800 will fill more than the active site of an enzyme. 

Here is an image of one of the ligands from the search that we're going to learn to download. It is identified in the Protein Data Bank as 13U. As a group we will look at some of the features that make this ligand interesting. 

<img src="images/13U.png" style="margin-left:auto; margin-right:auto"/>

The last statement in the previous cell

`len(resultL)`

tells us how many PDB entries have ligands of that size. The default return item for the query is `structure`, which provides the four character alphanumeric entry for the full structure in the PDB. We want to identify and download the ligands that are bound to these PDB structures, so we need to switch return types. 

To get the ligand, instead of returning the `structure`, we will request a return type of `mol_definition` which will then return the three character alphanumeric entry for the ligand. Other possible return types are `polymer entities`, `assemblies`, and `non-polymer entities`.


In [3]:
molResultL = list(query("mol_definition"))
print("There are",len(molResultL), "ligands for EC Number", ECnumber, "in this list. Here is a list of the first 10 ligands.")
molResultL[0:10]

There are 112 ligands for EC Number 3.4.21.4 in this list. Here is a list of the first 10 ligands.


['0CA', '0CB', '0KV', '0ZG', '0ZW', '0ZX', '0ZY', '10U', '11U', '12U']

### Where can we go to download the ligand files?

To download the files for ligands bound to trypsin in the RCSB PDB, execute the two cells above for finding the trypsin ligands. This will reset the results to the ones we want.

Once this is done, the next step is to determine exactly what we want to download. These ligand files in the PDB are avaiable for download in several formats. A full list and description can be found in the [Small Molecule File table](https://www.rcsb.org/docs/programmatic-access/file-download-services#small-molecule-files) on the [RCSB PDB File Download Services page](https://www.rcsb.org/docs/programmatic-access/file-download-services), which is pasted in here.

![Small molecule file formats that can be downloaded from the RCSB PDB](images/SmallMoleculeFilesTable.png "a title")

From this table, we want the ligand files in mol2 format, which we will later convert to another format called `pdbqt` for docking.

### How do we download the ligand files?

There are several options for downloading files - we will use a Python libary called `requests`. In the following cells, we will import the library, `requests`, download a single file from the RCSB PDB using the `requests.get` function, and check to make sure the file downloaded properly to the ligands folder. If that is successful, we'll use a `for` loop to download all of the files from our `molResultL` list to the ligands folder.

In [4]:
import requests  # to enable us to pull files from the PDB
import os        # to enable us to create a directory to store the files

In [5]:
# Download one of the files from our list: 11U.mol2

res11U_mol2 = requests.get('https://files.rcsb.org/ligands/download/11U_ideal.mol2')

In [6]:
# check to see that the file downloaded properly. A status code of 200 means everything is okay.

res11U_mol2.status_code

200

In [7]:
# To really be sure, let's look at the file one line at a time. First we write the downloaded content to a file.

# make a ligands folder for our results
os.makedirs("ligands", exist_ok=True)

with open("ligands/res11U.mol2", "w+") as file:
    file.write(res11U_mol2.text)

In [8]:
# Now we use these commands to read the file and make sure it downloaded properly. As an alternative, we
# could go to the ligands folder in our Jupyter desktop and click on res11U.mol2 to make sure it looks correct.

file1 = open('ligands/res11U.mol2', 'r')
file_text = file1.read() # This reads in the file as a string.

print(file_text)

@<TRIPOS>MOLECULE
11U
   59    61     0     0     0
SMALL
NO_CHARGES

@<TRIPOS>ATOM
      1 C1          2.4220    0.4070    0.3360 C.2       1 11U_ideal         0.0000
      2 O1          2.0060   -0.6420    0.7800 O.2       1 11U_ideal         0.0000
      3 C2          3.8690    0.5350   -0.0630 C.3       1 11U_ideal         0.0000
      4 N1          4.5590   -0.7380    0.1810 N.3       1 11U_ideal         0.0000
      5 C3          5.9760   -0.6510   -0.1970 C.3       1 11U_ideal         0.0000
      6 C4          6.7790   -0.0680    0.9670 C.3       1 11U_ideal         0.0000
      7 C5          8.2550    0.0240    0.5730 C.3       1 11U_ideal         0.0000
      8 C6          8.7810   -1.3740    0.2400 C.3       1 11U_ideal         0.0000
      9 C7          7.9780   -1.9570   -0.9250 C.3       1 11U_ideal         0.0000
     10 C8          6.5020   -2.0480   -0.5310 C.3       1 11U_ideal         0.0000
     11 N2          1.5890    1.4580    0.1990 N.am      1 11U_ideal        

### Downloading all of the ligands using a for loop

Now that we know that our process functions, we will use a `for` loop to download the entire list of ligands (all 112) in a single cell. Here are the steps we will take:

1. Define a variable, baseUrl, for the URL where the ligand files are located. The URL only lacks the specific name of the ligand file.
2. Set up a `for` loop to go through each of the items (as ChemID) in the molResultL list that was generated above.
3. Assign the filename based on a variable (the 3-letter name of the ligand as ChemID followed by \_ideal.mol2) to the variable cFile.
4. Assign the full URL (as cFileUrl) that we want to use to download the data from the RCSB PDB API. Notice that the URL will consist of the baseUrl (defined in the first line of the cell) followed by the name of the file we just defined, which is now assigned to the variable, cFile. 
5. Tell the notebook that we want the file (CFileLocal) to be written to the ligands folder, using the os.path command.
6. Use the API call via `requests.get` to download the data from the RCSB PDB.
7. Write the file using the `with open` function.

If all goes according to plan, this should download all of the ligands on our list to the ligands folder.


In [9]:
baseUrl = "https://files.rcsb.org/ligands/download/"

for ChemID in molResultL:
    cFile = f"{ChemID}_ideal.mol2"
    cFileUrl = baseUrl + cFile
    cFileLocal = "ligands/" + cFile
    response = requests.get(cFileUrl)
    with open(cFileLocal, "w+") as file:
        file.write(response.text)

### Selected ligands

For our next notebook, we are going to select and modify one of the ligands from the list. Any of them could be used, but we will be using [13U: N-cyclooctylglycyl-N-(4-carbamimidoylbenzyl)-L-prolinamide](https://www.rcsb.org/ligand/13U).

### Exercise

To go a bit deeper with these tools, use the [BRENDA Enzyme Database](https://www.brenda-enzymes.org/) to find the EC# for alcohol dehydrogenase (or look for an enzyme that interests you). How many structures have ligands with molecular weights between 400 and 700? How many unique ligands are bound to these structures? 

Note: You can enter only the upper levels of an EC Class to identify more ligands. This exercise can be repeated with any EC#. If you have time, try a broader search where you use only 2 or 3 levels, e.g., 3.4 or 3.4.21, and see what you find.

In [ ]:
### Solution

ECnumber = "1.1.1.1"     # We will use this variable again later

q1 = attrs.rcsb_polymer_entity.rcsb_ec_lineage.id == ECnumber  # looking for trypsins
q2 = attrs.chem_comp.formula_weight >= 400                      # setting the lower limit for molecular weight
q3 = attrs.chem_comp.formula_weight <= 700                     # setting the upper limit for molecular weight

query = q1 & q2 & q3              # combining the three queries into one

ResultL = list(query("entry"))
molResultL = list(query("mol_definition"))
print("There are",len(ResultL), "structures from EC Number", ECnumber, "that have bound ligands with molecular weights between 400 and 700).")
print("There are",len(molResultL), "unique ligands for structures with EC Number", ECnumber, "in this list. Here is a list of the", len(molResultL), "ligands.")
molResultL